<a href="https://colab.research.google.com/github/dele22922/tarea3_feat_eng/blob/main/EjemploPulpTransp_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You want to minimize the cost of shipping goods from 2 different warehouses to 4 different customers. Each warehouse has a limited supply and each customer has a certain demand. We need to fulfil the demand of the customers by shipping products from given warehouses such that the overall cost of shipping is minimum and we are also able to satisfy the customer demands using limited supply available with each warehouse.
![image.png](attachment:image.png)

The customer demands and the warehouse availability is as follows.

![image-2.png](attachment:image-2.png)

In [2]:
!pip install pulp
from pulp import *
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 63.9 MB/s eta 0:00:00


In [3]:
n_warehouses = 2
n_customers = 4

# Cost Matrix
cost_matrix = np.array([[1, 3, 0.5, 4],
                       [2.5, 5, 1.5, 2.5]])
# Demand Matrix
cust_demands = np.array([35000, 22000, 18000, 30000])

# Supply Matrix
warehouse_supply = np.array([60000, 80000])

In [ ]:
model = LpProblem("Supply-Demand-Problem", LpMinimize)

In [ ]:
variable_names = [str(i)+str(j) for j in range(1, n_customers+1) for i in range(1, n_warehouses+1)]
variable_names.sort()
print("Variable Indices:", variable_names)

Variable Indices: ['11', '12', '13', '14', '21', '22', '23', '24']


In [ ]:
DV_variables = LpVariable.matrix("X", variable_names, cat = "Integer", lowBound= 0 )
allocation = np.array(DV_variables).reshape(2,4)
print("Decision Variable/Allocation Matrix: ")
print(allocation)

Decision Variable/Allocation Matrix: 
[[X_11 X_12 X_13 X_14]
 [X_21 X_22 X_23 X_24]]


In [ ]:
obj_func = lpSum(allocation*cost_matrix)
print(obj_func)
model +=  obj_func
print(model)

X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24
Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer



In [ ]:
#Supply Constraints
for i in range(n_warehouses):
    print(lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i])
    model += lpSum(allocation[i][j] for j in range(n_customers)) <= warehouse_supply[i] , "Supply Constraints " + str(i)

X_11 + X_12 + X_13 + X_14 <= 60000
X_21 + X_22 + X_23 + X_24 <= 80000


In [ ]:
# Demand Constraints
for j in range(n_customers):
    print(lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j])
    model += lpSum(allocation[i][j] for i in range(n_warehouses)) >= cust_demands[j] , "Demand Constraints " + str(j)

X_11 + X_21 >= 35000
X_12 + X_22 >= 22000
X_13 + X_23 >= 18000
X_14 + X_24 >= 30000


In [ ]:
print(model)

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
SUBJECT TO
Supply_Constraints_0: X_11 + X_12 + X_13 + X_14 <= 60000

Supply_Constraints_1: X_21 + X_22 + X_23 + X_24 <= 80000

Demand_Constraints_0: X_11 + X_21 >= 35000

Demand_Constraints_1: X_12 + X_22 >= 22000

Demand_Constraints_2: X_13 + X_23 >= 18000

Demand_Constraints_3: X_14 + X_24 >= 30000

VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer



In [ ]:
#model.solve()
model.solve(PULP_CBC_CMD())

status =  LpStatus[model.status]

print(status)

Optimal


In [ ]:
print("Total Cost:", model.objective.value())

# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

Total Cost: 200000.0
X_11 = 35000.0
X_12 = 22000.0
X_13 = 3000.0
X_14 = 0.0
X_21 = 0.0
X_22 = 0.0
X_23 = 15000.0
X_24 = 30000.0


In [ ]:
# Warehouse 1 and Warehouse 2 required capacity

for i in range(n_warehouses):
    print("Warehouse ", str(i+1))
    print(lpSum(allocation[i][j].value() for j in range(n_customers)))

Warehouse  1
60000.0
Warehouse  2
45000.0
